In [159]:
import pandas as pd
import numpy as np
import re
import difflib

Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

In [160]:
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)

       LIBELLE_ARRET  00:00-01:00  01:00-02:00  02:00-03:00  03:00-04:00  \
0           abbesses        10.18         3.20         0.02         0.00   
1              ablon         0.26         0.03         0.00         0.02   
2      acheres grand         0.78          NaN          NaN          NaN   
3      acheres ville         0.76         0.31         0.13         0.09   
4      aeroport cdg1         0.76         0.14         0.01         0.04   
..               ...          ...          ...          ...          ...   
751           vosves          NaN          NaN          NaN          NaN   
752  vulaines sur se         0.11          NaN          NaN          NaN   
753           wagram         3.63         1.43         0.10         0.00   
754           yerres         0.73         0.11         0.01         0.05   
755   yvris noisy gd         0.20         0.04          NaN          NaN   

     04:00-05:00  05:00-06:00  06:00-07:00  07:00-08:00  08:00-09:00  ...  \
0         

/tmp/ipykernel_1830/2722053647.py:3: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  données_validation = pd.read_csv(url, delimiter=';')


Tableau avec station, ligne, correspondances...

In [161]:
tableau_fréquentation = pd.read_excel('https://data.ratp.fr/api/explore/v2.1/catalog/datasets/trafic-annuel-entrant-par-station-du-reseau-ferre-2021/exports/xlsx?lang=fr&timezone=Europe%2FBerlin&use_labels=true')

print(tableau_fréquentation)

     Rang Réseau                 Station    Trafic Correspondance_1  \
0       7  Métro              REPUBLIQUE  11079708                3   
1       9  Métro              LA DEFENSE   9256802                1   
2      13  Métro          HOTEL DE VILLE   7251729                1   
3      30  Métro  VILLEJUIF-LOUIS ARAGON   5218070                7   
4      40  Métro            PALAIS-ROYAL   4822599                1   
..    ...    ...                     ...       ...              ...   
366    33    RER        NOGENT-SUR-MARNE   2358276                A   
367    43    RER                 BAGNEUX   1742783                B   
368    44    RER   LE PARC DE SAINT-MAUR   1617366                A   
369    54    RER               PALAISEAU    694838                B   
370    60    RER       FONTAINE-MICHALON    473408                B   

    Correspondance_2 Correspondance_3  Correspondance_4  Correspondance_5  \
0                  5                8               9.0              1

In [162]:
tableau_fréquentation = tableau_fréquentation[tableau_fréquentation['Réseau'] == 'Métro']
tableau_frequentation = tableau_fréquentation.drop_duplicates(subset='Station')
display(tableau_fréquentation)

,Rang,Réseau,Station,Trafic,Correspondance_1,Correspondance_2,Correspondance_3,Correspondance_4,Correspondance_5,Ville,Arrondissement pour Paris
0,7,Métro,REPUBLIQUE,11079708,3,5,8,9.0,11.0,Paris,11.0
1,9,Métro,LA DEFENSE,9256802,1,NaN,NaN,NaN,NaN,Puteaux,NaN
2,13,Métro,HOTEL DE VILLE,7251729,1,11,NaN,NaN,NaN,Paris,4.0
3,30,Métro,VILLEJUIF-LOUIS ARAGON,5218070,7,NaN,NaN,NaN,NaN,Villejuif,NaN
4,40,Métro,PALAIS-ROYAL,4822599,1,7,NaN,NaN,NaN,Paris,1.0
...,...,...,...,...,...,...,...,...,...,...,...
357,286,Métro,PICPUS,931602,6,NaN,NaN,NaN,NaN,Paris,12.0
358,287,Métro,TEMPLE,891858,3,NaN,NaN,NaN,NaN,Paris,2.0
359,289,Métro,QUAI DE LA RAPEE,798728,5,NaN,NaN,NaN,NaN,Paris,12.0
360,299,Métro,DANUBE,400157,7bis,NaN,NaN,NaN,NaN,Paris,19.0


In [163]:
colonnes_a_supprimer = ['Réseau', 'Ville', 'Arrondissement pour Paris']

tableau_fréquentation = tableau_fréquentation.drop(colonnes_a_supprimer, axis=1)

In [164]:
# Liste des colonnes à modifier
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Convertir les colonnes spécifiées en type chaîne (si elles ne le sont pas déjà)
tableau_fréquentation[colonnes_correspondance] = tableau_fréquentation[colonnes_correspondance].astype(str)

# Supprimer les lettres 'A' et 'B' des colonnes spécifiées dans tableau_fréquentation
for colonne in colonnes_correspondance:
    tableau_fréquentation[colonne] = tableau_fréquentation[colonne].str.replace('A', '').str.replace('B', '')

PROBLÈME À RÉGLER : CREER UNE COLONNE QUI COMPTE LE NOMBRE DE LIGNES

In [165]:
# Liste des colonnes à analyser
colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

# Calcul du nombre de cases non vides pour chaque ligne
tableau_fréquentation['nb_correspondance'] = tableau_fréquentation[colonnes_correspondance].count(axis=1)

In [166]:
# Diviser les valeurs de la colonne "Trafic" par les valeurs de la colonne "nb_correspondance"
tableau_fréquentation['Trafic'] = tableau_fréquentation['Trafic'] / tableau_fréquentation['nb_correspondance']
print(tableau_fréquentation)

     Rang                 Station     Trafic Correspondance_1  \
0       7              REPUBLIQUE  2215941.6                3   
1       9              LA DEFENSE  1851360.4                1   
2      13          HOTEL DE VILLE  1450345.8                1   
3      30  VILLEJUIF-LOUIS ARAGON  1043614.0                7   
4      40            PALAIS-ROYAL   964519.8                1   
..    ...                     ...        ...              ...   
357   286                  PICPUS   186320.4                6   
358   287                  TEMPLE   178371.6                3   
359   289        QUAI DE LA RAPEE   159745.6                5   
360   299                  DANUBE    80031.4             7bis   
361   303       PRE-SAINT-GERVAIS    56525.2             7bis   

    Correspondance_2 Correspondance_3 Correspondance_4 Correspondance_5  \
0                  5                8              9.0             11.0   
1                nan              nan              nan              n

Création d'un tableau avec la moyenne sur la période des JO en 2022 du nombre de validation hebdomadire à chaque station

In [167]:
#Nettoyage de données sur le nombre de validations: on obtient le nombre de validation moyen par jour sur 1 mois précis

import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url2 = '/workspaces/metro-JO/Données importées/2022_S2_NB_FER.csv'
données_validation2= pd.read_csv(url2)
données_validation2 = pd.read_csv(url2, delimiter=';')

# Exemple avec gestion de formats flexibles
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], errors='coerce', format="%d/%m/%Y")
donnees_mai = données_validation2.loc[(données_validation2['JOUR'].dt.month == 5)]
donnees_sum = donnees_mai.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Conversion de la colonne 'JOUR' en datetime
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrer les données pour les dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Grouper par 'JOUR', 'LIBELLE_ARRET' et 'CATEGORIE_TITRE' et calculer la somme des 'NB_VALD'
donnees_aggreg = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET', 'CATEGORIE_TITRE'])['NB_VALD'].sum().reset_index()

# Conversion de la colonne 'JOUR' en datetime si ce n'est pas déjà fait
données_validation2['JOUR'] = pd.to_datetime(données_validation2['JOUR'], format='%d/%m/%Y')

# Filtrage des dates spécifiques
donnees_filtrees = données_validation2[(données_validation2['JOUR'] >= '2022-07-24') & (données_validation2['JOUR'] <= '2022-08-11')]

# Somme de NB_VALD par JOUR et par LIBELLE_ARRET
somme_par_jour_arret = donnees_filtrees.groupby(['JOUR', 'LIBELLE_ARRET'])['NB_VALD'].sum().reset_index()

moyenne_par_arret = somme_par_jour_arret.groupby('LIBELLE_ARRET')['NB_VALD'].mean().reset_index()

# Normalisation des libellés
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.lower().str.strip()

# Somme des valeurs de NB_VALD pour les libellés normalisés
moyenne_par_arret = moyenne_par_arret.groupby('LIBELLE_ARRET')['NB_VALD'].sum().reset_index()

# Suppression des doublons
moyenne_par_arret = moyenne_par_arret.drop_duplicates(subset=['LIBELLE_ARRET'])

display(moyenne_par_arret)

,LIBELLE_ARRET,NB_VALD
0,abbesses,2277.315789
1,ablon,1101.105263
2,acheres grand,51.388889
3,acheres ville,3012.052632
4,aeroport cdg1,5050.947368
...,...,...
733,vosves,14.894737
734,vulaines sur se,42.894737
735,wagram,3268.157895
736,yerres,2743.000000


Création d'un dictionnaire contenant les fréquentations de chaque station de chaque ligne --> PROBLÈME À RÉGLER : CERTAINES CASES VIDES

In [168]:
numéro = [1, 2, 3, '3bis', 4, 5, 6, 7, '7bis', 8, 9, 10, 11, 12, 13, 14]

# Création des tableaux pour chaque élément de numéro
tableaux = {}

colonnes_correspondance = ['Correspondance_1', 'Correspondance_2', 'Correspondance_3', 'Correspondance_4', 'Correspondance_5']

for k in numéro:
    # Convertir k en string car il peut contenir des valeurs non numériques
    k_str = str(k)
    
    # Filtrer les lignes où la valeur k est présente dans les colonnes spécifiées
    filtered_rows = tableau_fréquentation[tableau_fréquentation[colonnes_correspondance].apply(lambda x: k_str in x.values, axis=1)]
    
    # Stocker les lignes filtrées dans un tableau nommé "ligne numéro[k]"
    tableaux[f'ligne numéro[{k_str}]'] = filtered_rows

# Parcourir chaque tableau dans le dictionnaire et supprimer les colonnes spécifiées
for tableau_nom, tableau in tableaux.items():
    tableaux[tableau_nom] = tableau.drop(columns=colonnes_correspondance)
    tableau = tableau.drop_duplicates(subset='Station')

# Parcourir chaque tableau dans le dictionnaire
for tableau_nom, tableau in tableaux.items():
    # Calculer la fréquentation totale pour chaque tableau
    fréquentation_totale_ligne = tableau['Trafic'].sum()

    # Ajouter la variable fréquentation_totale à chaque tableau
    tableaux[tableau_nom]['fréquentation_totale_ligne'] = fréquentation_totale_ligne
    tableau['Part du trafic de la station par rapport à toute la ligne'] = tableau['Trafic'] / tableau['fréquentation_totale_ligne']

# Supposons que la clé pour la correspondance soit 'LIBELLE_ARRET' dans le tableau moyenne_par_arret
cle_correspondance = 'LIBELLE_ARRET'

# Nettoyage des espaces et uniformisation de la casse
moyenne_par_arret['LIBELLE_ARRET'] = moyenne_par_arret['LIBELLE_ARRET'].str.strip().str.lower()
for tableau_nom, tableau in tableaux.items():
    tableau['Station'] = tableau['Station'].str.strip().str.lower()

    # Fusion en utilisant les noms de stations nettoyés
    merged_table = pd.merge(tableau, moyenne_par_arret, left_on='Station', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD de moyenne_par_arret
    merged_table = merged_table.rename(columns={'NB_VALD': 'fréquentation moyenne hors JO'})
    
    tableaux[tableau_nom] = merged_table

In [169]:
#compléter avec les valeurs manquantes

# Méthode pour trouver la correspondance la plus proche
def find_closest_match(target, candidates):
    closest_match = difflib.get_close_matches(target, candidates, n=1, cutoff=0.4)  # Ajustez le seuil selon vos besoins
    return closest_match[0] if closest_match else None

for k in numéro:
    k_str = str(k)
    # Accéder aux tableaux spécifiques
    current_table = tableaux[f'ligne numéro[{k_str}]']
    
    # Filtrer les lignes avec des valeurs manquantes dans la colonne 'LIBELLE_ARRET'
    missing_data = current_table[current_table['LIBELLE_ARRET'].isnull()]
    
    # Remplir les valeurs manquantes
    for index, row in missing_data.iterrows():
        closest_match = find_closest_match(row['Station'], moyenne_par_arret['LIBELLE_ARRET'])
        
        if closest_match:
            tableaux[f'ligne numéro[{k_str}]'].at[index, 'LIBELLE_ARRET'] = closest_match



In [170]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner avec le tableau moyenne_par_arret pour obtenir les valeurs NB_VALD associées
    merged_with_nb_vald = pd.merge(current_table, moyenne_par_arret[['LIBELLE_ARRET', 'NB_VALD']],
                                   left_on='LIBELLE_ARRET', right_on='LIBELLE_ARRET', how='left')
    
    # Renommer la colonne NB_VALD pour plus de clarté
    merged_with_nb_vald = merged_with_nb_vald.rename(columns={'NB_VALD': 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'})

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = merged_with_nb_vald

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Fusionner les données avec le tableau contenant les nouvelles colonnes
    merged_table = pd.merge(current_table, pivot_table, on='LIBELLE_ARRET', how='left')

    # Mettre à jour le tableau dans le dictionnaire avec les nouvelles colonnes fusionnées
    tableaux[f'ligne numéro[{k_str}]'] = merged_table

In [171]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Supprimer les colonnes 'fréquentation moyenne hors JO' et 'LIBELLE_ARRET' si elles existent
    columns_to_drop = ['fréquentation moyenne hors JO', 'LIBELLE_ARRET']
    current_table = current_table.drop(columns=columns_to_drop, errors='ignore')

    # Mettre à jour le tableau dans le dictionnaire
    tableaux[f'ligne numéro[{k_str}]'] = current_table



In [172]:
for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance']

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si les colonnes existent avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station' in current_table.columns and 'nb_correspondance' in current_table.columns:
        # Calcul de la nouvelle colonne
        current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station'] / current_table['nb_correspondance'] / 2

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table

for k in numéro:
    k_str = str(k)
    # Accéder à chaque tableau spécifique
    current_table = tableaux[f'ligne numéro[{k_str}]']

    # Vérifier si la colonne existe avant de créer la nouvelle colonne
    if 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne' in current_table.columns:
        # Calculer la somme de la colonne spécifique
        current_table['nombre_validations_totales_pour_la_ligne'] = current_table['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne'].sum()

        # Mettre à jour le tableau dans le dictionnaire
        tableaux[f'ligne numéro[{k_str}]'] = current_table
        # Afficher le tableau mis à jour
        print(f"Tableau 'ligne numéro[{k_str}]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :")
        display(tableaux[f'ligne numéro[{k_str}]'])


Tableau 'ligne numéro[1]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,9,la defense,1851360.4,5,27894284.0,0.066371,77993.842105,7.07,1.51,0.01,...,103.14,108.83,84.18,53.23,26.30,14.93,14.40,15598.768421,7799.384211,138177.887449
1,13,hotel de ville,1450345.8,5,27894284.0,0.051994,15023.947368,12.16,4.87,0.09,...,60.08,62.00,53.19,32.02,19.18,19.01,18.49,3004.789474,1502.394737,138177.887449
2,40,palais-royal,964519.8,5,27894284.0,0.034578,12183.368421,9.30,2.52,0.09,...,70.75,67.08,42.63,27.29,21.24,20.44,20.58,2436.673684,1218.336842,138177.887449
3,86,concorde,680243.8,5,27894284.0,0.024386,9636.526316,10.30,2.87,0.08,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.305263,963.652632,138177.887449
4,172,argentine,415842.4,5,27894284.0,0.014908,4396.631579,6.64,2.50,0.16,...,41.96,46.19,39.54,23.87,13.05,10.02,9.98,879.326316,439.663158,138177.887449
5,187,champs-elysees-clemenceau,381801.0,5,27894284.0,0.013687,4301.315789,15.47,5.89,0.09,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.263158,430.131579,138177.887449
6,194,tuileries,371910.4,5,27894284.0,0.013333,4690.315789,10.76,2.14,0.03,...,59.73,61.08,51.92,32.26,25.46,23.30,22.92,938.063158,469.031579,138177.887449
7,11,bastille,1613848.6,5,27894284.0,0.057856,15795.736842,18.39,8.33,0.40,...,49.12,48.68,39.54,24.82,22.50,28.78,25.92,3159.147368,1579.573684,138177.887449
8,18,franklin d. roosevelt,1234766.0,5,27894284.0,0.044266,13231.894737,12.37,3.18,0.04,...,57.97,65.83,58.93,37.06,25.12,20.40,17.47,2646.378947,1323.189474,138177.887449
9,20,nation,1210159.4,5,27894284.0,0.043384,19465.157895,15.32,3.51,0.30,...,90.17,88.67,79.27,49.96,33.56,27.09,22.85,3893.031579,1946.515789,138177.887449


Tableau 'ligne numéro[2]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,123,victor hugo,560208.2,5,16877281.0,0.033193,5284.210526,7.13,1.43,0.03,...,48.86,48.08,43.57,22.99,12.69,12.26,12.33,1056.842105,528.421053,70635.936842
1,137,alexandre dumas,495321.6,5,16877281.0,0.029348,4649.315789,6.21,2.27,0.09,...,43.01,39.28,33.30,22.87,14.42,12.51,10.12,929.863158,464.931579,70635.936842
2,20,nation,1210159.4,5,16877281.0,0.071703,19465.157895,15.32,3.51,0.30,...,90.17,88.67,79.27,49.96,33.56,27.09,22.85,3893.031579,1946.515789,70635.936842
3,26,barbes-rochechouart,1078187.8,5,16877281.0,0.063884,11939.052632,5.36,2.10,0.07,...,45.75,44.03,37.70,22.18,14.09,11.06,8.63,2387.810526,1193.905263,70635.936842
4,72,villiers,717275.2,5,16877281.0,0.042499,6554.789474,6.53,1.88,0.10,...,43.72,45.97,42.67,26.71,14.65,12.93,11.77,1310.957895,655.478947,70635.936842
5,77,pigalle,700366.2,5,16877281.0,0.041498,6591.263158,19.27,8.09,0.09,...,39.98,40.12,36.95,26.15,21.85,28.69,26.75,1318.252632,659.126316,70635.936842
6,85,anvers,684149.8,5,16877281.0,0.040537,6515.157895,10.19,3.06,0.02,...,48.60,42.74,37.55,24.20,19.35,25.98,19.82,1303.031579,651.515789,70635.936842
7,149,ternes,458451.2,5,16877281.0,0.027164,4832.631579,6.72,1.92,0.08,...,46.53,48.19,45.57,24.19,13.65,14.27,14.99,966.526316,483.263158,70635.936842
8,224,courcelles,316685.8,5,16877281.0,0.018764,3163.684211,4.81,1.88,0.08,...,53.72,52.95,45.88,25.35,11.32,9.70,8.31,632.736842,316.368421,70635.936842
9,232,funiculaire,305734.2,5,16877281.0,0.018115,4471.315789,5.63,1.22,NaN,...,47.08,42.86,37.92,31.95,27.11,20.99,13.78,894.263158,447.131579,70635.936842


Tableau 'ligne numéro[3]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


...

,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,86,concorde,680243.8,5,23051055.6,0.029510,9636.526316,10.30,2.87,0.08,...,66.58,69.70,54.94,27.15,19.30,18.36,17.88,1927.305263,963.652632,105667.292763
1,105,pasteur,605257.2,5,23051055.6,0.026257,6307.736842,7.42,2.12,0.13,...,43.50,44.12,35.49,20.42,12.83,11.10,10.56,1261.547368,630.773684,105667.292763
2,233,abbesses,302776.8,5,23051055.6,0.013135,2277.315789,10.18,3.20,0.02,...,45.59,43.24,37.00,23.36,19.02,19.36,18.35,455.463158,227.731579,105667.292763
3,280,trinite-d'estienne d'orves,210396.4,5,23051055.6,0.009127,3454.789474,12.18,6.56,0.22,...,51.19,54.76,49.49,33.64,26.85,21.44,16.85,690.957895,345.478947,105667.292763
4,291,saint-georges,150311.0,5,23051055.6,0.006521,1449.105263,9.83,3.71,0.10,...,47.79,42.88,34.41,21.11,16.17,26.32,17.00,289.821053,144.910526,105667.292763
5,296,assemblee nationale,122302.4,5,23051055.6,0.005306,1602.263158,14.18,8.04,0.10,...,65.30,64.65,41.95,25.79,16.80,15.93,18.23,320.452632,160.226316,105667.292763
6,2,saint-lazare,6625676.8,5,23051055.6,0.287435,127163.368421,8.62,0.10,0.01,...,57.36,63.81,51.34,34.47,21.65,16.09,13.86,25432.673684,12716.336842,105667.292763
7,2,saint-lazare,6625676.8,5,23051055.6,0.287435,127163.368421,8.06,1.80,0.14,...,85.45,92.97,77.22,51.26,27.60,21.59,14.81,25432.673684,12716.336842,105667.292763
8,55,marcadet-poissonniers,796401.0,5,23051055.6,0.034549,9417.421053,4.49,1.81,0.10,...,43.07,42.94,37.58,27.00,16.68,10.82,7.56,1883.484211,941.742105,105667.292763
9,67,convention,746950.0,5,23051055.6,0.032404,7157.157895,4.27,1.32,0.07,...,37.75,38.46,33.51,20.97,13.06,9.80,7.74,1431.431579,715.715789,105667.292763


Tableau 'ligne numéro[13]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,54,basilique de saint-denis,798279.0,5,16957848.6,0.047074,1889.263158,1.98,0.30,0.05,...,39.92,33.92,25.82,14.12,8.43,6.61,4.61,377.852632,188.926316,37710.673684
1,76,plaisance,704350.6,5,16957848.6,0.041535,7378.736842,4.13,1.31,0.03,...,39.72,38.27,32.66,22.15,12.53,8.94,7.01,1475.747368,737.873684,37710.673684
2,155,malakoff-plateau de vanves,448464.0,5,16957848.6,0.026446,4647.684211,2.93,1.02,0.09,...,39.62,41.65,31.21,18.21,9.74,7.87,5.92,929.536842,464.768421,37710.673684
3,158,guy moquet,441062.6,5,16957848.6,0.026009,5378.578947,5.08,2.09,0.06,...,37.81,36.78,33.77,20.83,12.24,9.86,8.41,1075.715789,537.857895,37710.673684
4,187,champs-elysees-clemenceau,381801.0,5,16957848.6,0.022515,4301.315789,15.47,5.89,0.09,...,62.33,67.18,52.39,31.77,25.55,24.32,23.10,860.263158,430.131579,37710.673684
5,189,les agnettes,378071.2,5,16957848.6,0.022295,4123.315789,2.65,0.77,0.04,...,34.57,31.37,25.58,17.10,10.05,7.16,5.00,824.663158,412.331579,37710.673684
6,198,la fourche,365854.2,5,16957848.6,0.021574,3468.052632,7.98,3.66,0.13,...,40.81,40.02,37.61,24.24,15.98,12.67,11.38,693.610526,346.805263,37710.673684
7,290,varenne,156539.4,5,16957848.6,0.009231,1845.105263,6.08,2.24,0.09,...,69.74,68.24,34.46,17.61,11.87,10.16,16.57,369.021053,184.510526,37710.673684
8,35,chatillon-montrouge,1006802.4,5,16957848.6,0.059371,11674.526316,2.28,0.61,0.02,...,41.77,41.56,29.17,16.87,9.27,6.52,5.61,2334.905263,1167.452632,37710.673684
9,39,mairie de saint-ouen,966162.0,5,16957848.6,0.056974,12017.736842,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,37710.673684


Tableau 'ligne numéro[14]' avec la nouvelle colonne 'moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction' :


,Rang,Station,Trafic,nb_correspondance,fréquentation_totale_ligne,Part du trafic de la station par rapport à toute la ligne,moyenne_NB_VALD_hebdo_période_jo_2022_dans_toute_la_station,00:00-01:00,01:00-02:00,02:00-03:00,...,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-00:00,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne,moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction,nombre_validations_totales_pour_la_ligne
0,39,mairie de saint-ouen,966162.0,5,15764240.2,0.061288,12017.736842,7.75,2.71,0.19,...,93.80,89.63,63.45,40.80,24.30,16.35,13.66,2403.547368,1201.773684,47415.801671
1,50,pont cardinet,833707.6,5,15764240.2,0.052886,9483.263158,3.67,1.64,0.11,...,43.22,45.32,37.99,23.12,14.08,7.68,6.32,1896.652632,948.326316,47415.801671
2,56,pyramides,792539.6,5,15764240.2,0.050275,8558.263158,6.84,2.03,0.13,...,63.20,67.67,51.92,33.49,24.30,22.48,18.01,1711.652632,855.826316,47415.801671
3,84,clichy saint-ouen,684170.4,5,15764240.2,0.043400,9117.842105,2.59,1.24,0.18,...,54.71,54.96,36.11,20.36,12.13,6.13,4.06,1823.568421,911.784211,47415.801671
4,6,bibliotheque,2220894.8,5,15764240.2,0.140882,6088.000000,6.46,1.35,0.06,...,54.02,56.83,44.83,26.99,18.84,11.58,8.76,1217.600000,608.800000,47415.801671
5,31,olympiades,1042919.0,5,15764240.2,0.066157,12997.142857,2.74,1.23,0.06,...,47.26,41.98,31.78,20.50,13.19,6.90,4.47,2599.428571,1299.714286,47415.801671
6,3,gare de lyon,5728095.0,5,15764240.2,0.363360,66037.947368,2.16,0.01,0.00,...,57.99,60.21,38.73,27.60,17.40,10.24,4.80,13207.589474,6603.794737,47415.801671
7,3,gare de lyon,5728095.0,5,15764240.2,0.363360,66037.947368,18.38,4.31,0.30,...,84.89,89.26,74.77,56.04,41.78,38.76,34.89,13207.589474,6603.794737,47415.801671
8,61,bercy,776842.4,5,15764240.2,0.049279,9023.947368,NaN,NaN,NaN,...,104.32,67.23,40.18,13.57,0.11,14.23,0.09,1804.789474,902.394737,47415.801671
9,61,bercy,776842.4,5,15764240.2,0.049279,9023.947368,10.59,2.39,0.22,...,43.36,44.68,36.17,35.22,24.11,25.58,36.50,1804.789474,902.394737,47415.801671
